# Análise de Consumo de Álcool por País

Este notebook analisa os dados de consumo de álcool **por pessoa** em diferentes países no ano de 2010, incluindo cerveja, destilados e vinho.

**Unidades:** Doses médias por pessoa (beer_servings, spirit_servings, wine_servings) e litros de álcool puro por pessoa por ano (total_litres_of_pure_alcohol).

**Fonte:** FiveThirtyEight, _"Dear Mona Followup: Where Do People Drink The Most Beer, Wine And Spirits?"_, 2014; World Health Organisation, 2010; Global Information System on Alcohol and Health (GISAH), 2010.

## 1. Importação de Bibliotecas

Para visualizações geográficas, utilizaremos também o **plotly** que permite criar mapas interativos.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## 2. Carregamento dos Dados

In [2]:
df_drinks = pd.read_csv("drinks.csv")

## 3. Reconhecimento e Exploração dos Dados


In [3]:
df_drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9


In [4]:
df_drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9
...,...,...,...,...,...
188,Venezuela,333,100,3,7.7
189,Vietnam,111,2,1,2.0
190,Yemen,6,0,0,0.1
191,Zambia,32,19,4,2.5


In [5]:
df_drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.7+ KB


In [6]:
df_drinks.describe()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,193.000000,193.000000,193.000000,193.000000
mean,106.160622,80.994819,49.450777,4.717098
std,101.143103,88.284312,79.697598,3.773298
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,1.000000,1.300000
50%,76.000000,56.000000,8.000000,4.200000
75%,188.000000,128.000000,59.000000,7.200000
max,376.000000,438.000000,370.000000,14.400000


## 4. Tratamento e Transformação de Dados

### 4.1 Criação de Categorias de Consumo

Criação de uma variável categórica para classificar os países de acordo com o nível de consumo de álcool puro **por pessoa**
(Utilizando como corte os valores nos quartis da amostra de dados).

**Critérios de categorização (litros/ano por pessoa):**
- **Very Low**: 0 - 1 litro/ano
- **Low**: 1 - 4 litros/ano
- **Medium**: 4 - 7 litros/ano
- **High**: 7 - 10 litros/ano
- **Very High**: acima de 10 litros/ano

In [7]:
bins = [0, 1, 4, 7, 10, float('inf')]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

df_drinks['consumption_category'] = pd.cut(df_drinks['total_litres_of_pure_alcohol'], bins=bins, labels=labels, include_lowest=True)

In [8]:
df_drinks['consumption_category'].value_counts().sort_index()

consumption_category
Very Low     44
Low          48
Medium       50
High         26
Very High    25
Name: count, dtype: int64

### 4.2 Verificação da Distribuição das Categorias

In [9]:
df_drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,consumption_category
0,Afghanistan,0,0,0,0.0,Very Low
1,Albania,89,132,54,4.9,Medium
2,Algeria,25,0,14,0.7,Very Low
3,Andorra,245,138,312,12.4,Very High
4,Angola,217,57,45,5.9,Medium
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,High
189,Vietnam,111,2,1,2.0,Low
190,Yemen,6,0,0,0.1,Very Low
191,Zambia,32,19,4,2.5,Low


## 5. Análise Exploratória Visual (EDA Gráfica)

Análise visual da distribuição do consumo de bebidas alcoólicas e possíveis padrões entre as variáveis.



### 5.1 Mapa-Múndi de Categorias de Consumo de Álcool

Visualização geográfica das categorias de consumo de álcool **por pessoa** por país. O mapa utiliza um esquema de cores para representar as diferentes faixas de consumo per capita.

In [10]:
# Mapeamento das categorias para valores numéricos (para escala de cores)
category_order = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
df_drinks['category_numeric'] = df_drinks['consumption_category'].cat.codes

# Criar o mapa coroplético
fig = px.choropleth(
    df_drinks,
    locations="country",
    locationmode='country names',
    color="consumption_category",
    hover_name="country",
    hover_data={
        'total_litres_of_pure_alcohol': ':.2f',
        'beer_servings': True,
        'wine_servings': True,
        'spirit_servings': True,
        'consumption_category': True,
        'category_numeric': False
    },
    color_discrete_sequence=px.colors.sequential.Reds,
    category_orders={'consumption_category': category_order},
    title='Categorias de Consumo de Álcool por País (2010) - Per Capita',
    labels={
        'consumption_category': 'Categoria de Consumo',
        'total_litres_of_pure_alcohol': 'Litros de Álcool Puro/Ano (por pessoa)',
        'beer_servings': 'Doses de Cerveja (por pessoa)',
        'wine_servings': 'Doses de Vinho (por pessoa)',
        'spirit_servings': 'Doses de Destilados (por pessoa)'
    }
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600,
    width=1000
)

fig.show()

### 5.2 Mapa-Múndi de Consumo Total de Álcool (Escala Contínua)

Visualização alternativa mostrando o consumo total de álcool puro **por pessoa** em uma escala de cores contínua, permitindo identificar nuances na distribuição geográfica do consumo per capita.

In [11]:
# Mapa com escala contínua de consumo
fig2 = px.choropleth(
    df_drinks,
    locations="country",
    locationmode='country names',
    color="total_litres_of_pure_alcohol",
    hover_name="country",
    hover_data={
        'total_litres_of_pure_alcohol': ':.2f',
        'beer_servings': True,
        'wine_servings': True,
        'spirit_servings': True,
        'consumption_category': True
    },
    color_continuous_scale='Reds',
    title='Consumo Total de Álcool Puro por País (2010) - Litros/Ano Per Capita',
    labels={
        'total_litres_of_pure_alcohol': 'Litros/Ano (por pessoa)',
        'beer_servings': 'Doses de Cerveja (por pessoa)',
        'wine_servings': 'Doses de Vinho (por pessoa)',
        'spirit_servings': 'Doses de Destilados (por pessoa)'
    }
)

fig2.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    height=600,
    width=1000
)

fig2.show()

### 5.2 